<h1 div class='alert alert-success'><center> TPS-Set: Feature Engineering</center></h1>

![](https://storage.googleapis.com/kaggle-competitions/kaggle/26480/logos/header.png?t=2021-04-09-00-57-05)

# <div class="alert alert-success">  0. IMPORTAÇÕES </div>

In [ ]:
import warnings
import random
import os
import gc
import shap

In [ ]:
import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt 
import seaborn           as sns
import joblib            as jb
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection   import train_test_split
from sklearn.preprocessing     import QuantileTransformer
from sklearn.cluster           import KMeans
from yellowbrick.cluster       import KElbowVisualizer, SilhouetteVisualizer
from scipy.stats.mstats        import winsorize
from sklearn.decomposition     import PCA
from sklearn.impute            import SimpleImputer
from sklearn.metrics           import silhouette_samples, silhouette_score
from sklearn.model_selection   import train_test_split, KFold, RepeatedStratifiedKFold, StratifiedKFold  
from sklearn                   import metrics
from prettytable               import PrettyTable

In [ ]:
import xgboost                 as xgb

## 0.2. Funções

In [ ]:
def jupyter_setting():
    
    %matplotlib inline
      
    #os.environ["WANDB_SILENT"] = "true" 
    #plt.style.use('bmh') 
    #plt.rcParams['figure.figsize'] = [20,15]
    #plt.rcParams['font.size']      = 13
     
    pd.options.display.max_columns = None
    #pd.set_option('display.expand_frame_repr', False)

    warnings.filterwarnings(action='ignore')
    warnings.simplefilter('ignore')
    warnings.filterwarnings('ignore')
    #warnings.filterwarnings(category=UserWarning)
    
    #pd.set_option('display.max_rows', 5)
    #pd.set_option('display.max_columns', 500)
    #pd.set_option('display.max_colwidth', None)

    icecream = ["#00008b", "#960018","#008b00", "#00468b", "#8b4500", "#582c00"]
    #sns.palplot(sns.color_palette(icecream))
    
    return icecream

icecream = jupyter_setting()

warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=UserWarning)
#warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

In [ ]:
def missing_zero_values_table(df):
        mis_val         = df.isnull().sum()
        mis_val_percent = round(df.isnull().mean().mul(100), 2)
        mz_table        = pd.concat([mis_val, mis_val_percent], axis=1)
        mz_table        = mz_table.rename(columns = {df.index.name:'col_name', 
                                                     0 : 'Valores ausentes', 
                                                     1 : '% de valores totais'})
        
        mz_table['Tipo de dados'] = df.dtypes
        mz_table                  = mz_table[mz_table.iloc[:,1] != 0 ]. \
                                     sort_values('% de valores totais', ascending=False)
        
        msg = "Seu dataframe selecionado tem {} colunas e {} " + \
              "linhas. \nExistem {} colunas com valores ausentes."
            
        print (msg.format(df.shape[1], df.shape[0], mz_table.shape[0]))
        
        return mz_table.reset_index()

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
        
    return df

In [ ]:
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall")
    plt.rcParams['font.size'] = 12
    plt.title('Precision Recall vs threshold')
    plt.xlabel('Threshold')
    plt.legend(loc="lower left")
    
    plt.grid(True)

In [ ]:
def plot_precision_vs_recall(precisions, recalls):
    plt.plot(recalls[:-1], precisions[:-1], "b-", label="Precision")
    
    plt.rcParams['font.size'] = 12
    plt.title('Precision vs recall')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    # plt.legend(loc="lower left")
    
    plt.grid(True)

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    fig, ax = plt.subplots()
    ax.plot(fpr, tpr, "r-", label=label)
    ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="--", c=".3")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.rcParams['font.size'] = 12
    plt.title('XGBR ROC curve for TPS 09')
    plt.xlabel('False Positive Rate (1 - Specificity)')
    plt.ylabel('True Positive Rate (Sensitivity)')
    plt.legend(loc="lower right")
    plt.grid(True)

## 0.3. Carregar Dados

In [ ]:
path = '../input/tabular-playground-series-sep-2021/'
#path = '../input/tabular-playground-series-sep-2021/'
#path = 'Data/'

In [ ]:
df1_train     = pd.read_csv(path + 'train.csv', index_col='id')
df1_test      = pd.read_csv(path + 'test.csv',index_col='id')
df_submission = pd.read_csv(path + 'sample_solution.csv')

df1_train.shape, df1_test.shape, df_submission.shape

In [ ]:
df1_test.head()

In [ ]:
df1_train = reduce_memory_usage(df1_train)
df1_test  = reduce_memory_usage(df1_test)

df1_train.shape, df1_test.shape

# <div class="alert alert-success"> 1. Feature Engineering </div>

In [ ]:
features_df = df1_train.columns.to_list()
features_df.remove('claim')

## 1.1. Step 01

A ideia de adicionar o recurso "n_missing" abaixo foi tirada deste [bloco notas](https://www.kaggle.com/maximkazantsev/tps-09-21-eda-lightgbm-with-folds) de por [BIZEN](https://www.kaggle.com/hiro5299834) .

In [ ]:
df1_train['fe_n_missing'] = df1_train[features_df].isna().sum(axis=1)
df1_test['fe_n_missing']  = df1_test[features_df].isna().sum(axis=1)

df1_train.shape, df1_test.shape

In [ ]:
features_df.append('fe_n_missing')

## 2.1. Step 02 
Tratamento de valores faltantes, para os valores faltantes vamos imputar a mediana. 

In [ ]:
imputer   = SimpleImputer(missing_values=np.nan, strategy='median')
df1_train = pd.DataFrame(imputer.fit_transform(df1_train), columns=df1_train.columns)
df1_test  = pd.DataFrame(imputer.fit_transform(df1_test),  columns=df1_test.columns)
gc.collect()

In [ ]:
missing_zero_values_table(df1_train)

In [ ]:
missing_zero_values_table(df1_test)

## 2.3. Step 03
Aqui vamos criar duas variáveis que indica os valores estão próximo de zero ou não. 

In [ ]:
df1_train['fe_f5_zero']  = (df1_train['f5'] < 0.02).astype(int)
df1_test['fe_f5_zero']   = (df1_test['f5'] < 0.02).astype(int)
df1_train['fe_f50_zero'] = (df1_train['f50'] < 0.02).astype(int)
df1_test['fe_f50_zero']  = (df1_test['f50'] < 0.02).astype(int)

Agora vamos criar algumas variáveis de estatística descritiva. 

In [ ]:
df1_train['fe_mean']    = df1_train.mean(axis=1)
df1_test['fe_mean']     = df1_test.mean(axis=1)

df1_train['fe_median']  = df1_train.median(axis=1)
df1_test['fe_median']   = df1_test.median(axis=1)

df1_train['fe_std']     = df1_train.std(axis=1)
df1_test['fe_std']      = df1_test.std(axis=1)

df1_train['fe_min']     = df1_train.min(axis=1)
df1_test['fe_min']      = df1_test.min(axis=1)

df1_train['fe_max']     = df1_train.max(axis=1)
df1_test['fe_max']      = df1_test.max(axis=1)

df1_train['fe_skew']    = df1_train.skew(axis=1)
df1_test['fe_skew']     = df1_test.skew(axis=1)

df1_train.shape, df1_test.shape

## 2.4. Step 04 
Neste step vamos criar uma nova variável com o algoritmo `kmeans`, para quem conhence o algoritmo vamos fazer uma clusterização, mas em primeiro lugar vamos criar algumas variáveis com uma PCD com as 40 principais componentes para a clusterização.

### 2.4.1. Scaler 

Vamos fazer o scaler dos datasets com o `QuatileTransforme` que teve melhor resultado na validação cruzada e no kaggle com AUC de 0.74479 com XGB, que se encontra no [notebook anterior](https://www.kaggle.com/rogeriodelfim/01-tps-set-ponto-de-partida-eda-e-linha-de-base?scriptVersionId=73995146).  

In [ ]:
%%time
scaler        = QuantileTransformer(output_distribution='normal', random_state=0)

df1_train_qt  = df1_train.copy().drop('claim', axis=1) 
cols_quantile = df1_train_qt.columns

df1_train_qt  = pd.DataFrame(scaler.fit_transform(df1_train_qt), columns=cols_quantile)
df1_test_qt   = pd.DataFrame(scaler.fit_transform(df1_test), columns=cols_quantile )

### 2.4.2. Gerar PCA
Nesta etapa vamos criar 20 componente principal que serão serão utilizadas na clusterização, normalmente as primeiras componentes de uma PCA resumem a maior parte da variância dos dados, no caso desse conjunto de dados vamos precisamos de 20 à 40 compontes para termos uma melhor explicabilidade da variância dos dados.  

In [ ]:
%%time
pca           = PCA(random_state=123)
df1_train_pca = pd.DataFrame(pca.fit_transform(df1_train_qt))

In [ ]:
fig = plt.figure(figsize=(12,8))
fig.subplots_adjust(wspace=.4, hspace=.4)

ax = fig.add_subplot(2, 1, 1)

ax.bar(range(1, 1+pca.n_components_),
       pca.explained_variance_ratio_,
       color='#FFB13F')

ax.set(xticks=[1, 2, 3, 4])
plt.yticks(np.arange(0, 1.1, 0.1))
plt.title('Variância explicada', fontsize=15)
plt.xlabel('Componentes principais', fontsize=13)
plt.ylabel('% de variância explicada', fontsize=13)

ax = fig.add_subplot(2, 1, 2)
ax.bar(range(1, 1+pca.n_components_),
       np.cumsum(pca.explained_variance_ratio_),
       color='#FFB13F')

ax.set(xticks=[1, 2, 3, 4])
plt.yticks(np.arange(0, 1.1, 0.1))
plt.title('Variância explicada cumulativa', fontsize=15)
plt.xlabel('Componentes principais', fontsize=13)
plt.ylabel('% de variância explicada', fontsize=13)
plt.show()

t = PrettyTable(['Componente',
                 'Variância explicada',
                 'Variância explicada cumulativa'])

principal_component = 1
cum_explained_var   = 0

for explained_var in pca.explained_variance_ratio_:
    cum_explained_var += explained_var
    t.add_row([principal_component, explained_var, cum_explained_var])
    principal_component += 1

print(t)

**`NOTA:`**

No primeiro gráfico observamos que os quatros primeiro compontes são pouco relevantes, com pouca explicabilidade, como podemos observar no segundo gráfico e na tabela de variância acumulada, sendo assim, vou selecionar 20 componeste para a clusterização, pois a explicabilidade chega à 35.45%.

In [ ]:
n_components  = 20
pca           = PCA(n_components=n_components, random_state=123)
pca_feats     = [f'feature_pca_{i}' for i in range(n_components)]
df1_train_pca = pd.DataFrame(pca.fit_transform(df1_train_qt), columns=pca_feats)
df1_test_pca  = pd.DataFrame(pca.fit_transform(df1_test_qt) , columns=pca_feats)

df1_train_pca.head()

Pronto agora vamos para a próxima etapa que é gerar a variável de clusterização. 

### 2.1.2. Clusterização

Nesta etapa vamos cria uma nova variável com utilização do algoritmo `kmeams`, que tem a finalidade de criar clusters, vamos utilizar as variáveis criadas no processo anterior `2.4.2. Gerar PCA`.

In [ ]:
df1_train  = df1_train.copy()
df1_test   = df1_test.copy()
df1_train.shape, df1_test.shape

In [ ]:
%%time 
gc.collect()

plt.figure(figsize=(12, 7))
visualizer_1 = KElbowVisualizer(KMeans(random_state=59), k=(2,12))
visualizer_1.fit(df1_train_pca);
visualizer_1.poof();

Como podemos observar no gráfico acima, precisamos criar um modelo para clusterização de 4 clusters, então vamos fazer isso agora nos dados de treino e replicar para os dados de teste. 

In [ ]:
gc.collect()

model_kmeans = KMeans(n_clusters=4, random_state=59)
model_kmeans.fit(df1_train_pca);

clusters_train = model_kmeans.predict(df1_train_pca)
clusters_test  = model_kmeans.predict(df1_test_pca)

df1_train['cluster'] = clusters_train
df1_test['cluster']  = clusters_test

df1_train.shape, df1_test.shape

Vamos salvar os dataset, caso seja necessário refazer o processo podemos partir desse ponto. 

In [ ]:
!mkdir Data
!mkdir Data/pkl
!mkdir Data/sumbmission
!mkdir model
!mkdir model/preds
!mkdir model/optuna

In [ ]:
jb.dump(df1_train,  "Data/pkl/df2_train.pkl.z")
jb.dump(df1_test,  "Data/pkl/df2_test.pkl.z")

In [ ]:
gc.collect()

# <div class="alert alert-success"> 2. Split Train/Test </div>

In [ ]:
df2_train = df1_train.copy()
df2_test  = df1_test.copy() 

# df2_train     = jb.load('./Data/pkl/df2_train.pkl.z')
# df2_test      = jb.load('./Data/pkl/df2_test.pkl.z')
# df_submission = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')

df2_train.shape, df2_test.shape

In [ ]:
X      = df2_train.drop(['claim'], axis=1)
y      = df2_train['claim']
cols   = X.columns
X_test = df2_test

X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size    = 0.2,
                                                      shuffle      = True, 
                                                      stratify     = y,
                                                      random_state = 0)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape 

# <div class="alert alert-success"> 3. Modelagem </div>

Parametros do algoritmo

In [ ]:
params = {'random_state': 0,          
          'predictor'   : 'gpu_predictor',
          'tree_method' : 'gpu_hist',
          'eval_metric' : 'auc'}

In [ ]:
model_xgb = xgb.XGBClassifier(**params)
model_xgb.fit(X_train, y_train)

y_hat_prob = model_xgb.predict_proba(X_valid)[:, 1]
y_hat      = (y_hat_prob >.5).astype(int)

fpr, tpr, thresholds = metrics.roc_curve(y_valid, y_hat_prob)
log_loss_     = metrics.log_loss(y_valid, y_hat_prob)                
f1_score_     = metrics.f1_score(y_valid, y_hat)        
auc_          = metrics.auc(fpr, tpr)    

print('AUC: {:.5f} - F1: {:.5f} - L. LOSS: {:.5f}'.format(auc_, f1_score_, log_loss_))
print('')

Obtivemos uma `AUC` de `0.81061`, isso que dizer que o acrescimo de novas variáveis nos dataset ajudou, porém precisams fazer uma validação mais robusta (validação cruzada) em vez de uma separação simples (Holdout), na validação cruzada vamos ter uma estimativa mais consistente.  

In [ ]:
print(metrics.classification_report(y_valid, (y_hat_prob >.35).astype(int)))

In [ ]:
metrics.plot_confusion_matrix(model_xgb, X_valid, y_valid, cmap='inferno')
plt.title('Confusion matrix')
plt.grid(False)
plt.show()

In [ ]:
precisions, recalls, thresholds = metrics.precision_recall_curve(y_valid, (y_hat_prob >.45).astype(int))

plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.show()

In [ ]:
plot_precision_vs_recall(precisions, recalls)
plt.show()

In [ ]:
threshold = .37
y_hat_threshold = (y_hat_prob>threshold).astype(int)
fpr, tpr, thresholds = metrics. roc_curve(y_valid, y_hat_threshold)

plot_roc_curve(fpr, tpr, label="XGB")
plt.show()

print('F1-score: {:2.5f}'.format(metrics.f1_score(y_valid, y_hat)))
print('F1-score: {:2.5f} threshold({:2.2f})'.format(metrics.f1_score(y_valid, y_hat_threshold), threshold))


## 4.1. Validação Cruzada

In [ ]:
%%time
X_train     = df2_train.drop(['claim'], axis=1)
y_train     = df2_train['claim']

params = {'random_state': 0,          
          'predictor'   : 'gpu_predictor',
          'tree_method' : 'gpu_hist',
          'eval_metric' : 'auc'}

gc.collect()

X_train_oof_shap = np.zeros((X_train.shape[0],X_train.shape[1]+1))
feature          = X_test.columns.to_list()
scalers          = [#QuantileTransformer(output_distribution='uniform', random_state=0),
                    QuantileTransformer(output_distribution='normal' , random_state=0)]

feature.append('claim')

X_ts = pd.DataFrame(scaler.fit_transform(X_test.copy()), columns=X_test.columns)

SEED = 59 

gc.collect()

X_train_oof_shap = np.zeros((X_train.shape[0],X_train.shape[1]+2))
feature          = X_test.columns.to_list()
scalers          = [#QuantileTransformer(output_distribution='uniform', random_state=0),
                    QuantileTransformer(output_distribution='normal' , random_state=0)]

feature.append('claim')

In [ ]:
%%time

for scaler in scalers: 

    FOLDS               = 5
    df_submission.claim = 0
    auc                 = []
    lloss               = []
    f1                  = []
    model_feature_imp   = 0 
    scale_distribution  = str.upper(scaler.get_params(True)['output_distribution']) 
    
    kfold               = KFold(n_splits = FOLDS, random_state = 0, shuffle = True)
 
    print('='*80)
    print('Scaler: {} => {}'.format(scaler, scale_distribution))
    print('='*80)

    cols_model_base = ['fe_f50_zero', 'fe_f5_zero', 'fe_n_missing', 'cluster', 
                       'fe_std', 'f105', 'f102', 'f22', 'f79', 'f106', 'f71', 
                       'f77', 'f69', #'f38','f57','f40', 'f1', 'f26', 'f82'
                      ]
    
    for i, (train_idx, test_idx) in enumerate(kfold.split(X_train)):

        i+=1
        
        X_tr, y_tr = X_train.iloc[train_idx], y_train.iloc[train_idx]
        X_vl, y_vl = X_train.iloc[test_idx], y_train.iloc[test_idx]
        
        X_tr = pd.DataFrame(scaler.fit_transform(X_tr), columns=cols) 
        X_vl = pd.DataFrame(scaler.fit_transform(X_vl), columns=cols)                
        
        # # Treinar o modelo baseline
        model_base = xgb.XGBClassifier(**params)
        model_base.fit(X_tr[cols_model_base], y_tr)
        
        threshold = .5
        
        y_hat_prob_tr_mb = (model_base.predict_proba(X_tr[cols_model_base])[:, 1]>threshold).astype(int) 
        y_hat_prob_vl_mb = (model_base.predict_proba(X_vl[cols_model_base])[:, 1]>threshold).astype(int) 
        y_hat_prob_ts_mb = (model_base.predict_proba(X_ts[cols_model_base])[:, 1]>threshold).astype(int) 
        
        X_tr['fe_md_baseline'] = y_hat_prob_tr_mb 
        X_vl['fe_md_baseline'] = y_hat_prob_vl_mb
        X_ts['fe_md_baseline'] = y_hat_prob_ts_mb
        
        params_xgb = {#'n_estimators'    : 500,
                      'max_depth'        : 7,
                      'min_child_weight' : 40,
                      'subsample'        : 0.662776837991846,
                      'colsample_bynode' : 0.7392362752914446,
                      'learning_rate'    : 0.18833941612449953,
                      'colsample_bytree' : 0.6094875461582554,
                      'reg_lambda'       : 10,
                      'reg_alpha'        : 42,
                      'eta'              : 0.07162137592923906,
                      'alpha'            : 0.5197749697495789}

        # Treinar XGB com a variável fe_md_baseline 
        model = xgb.XGBClassifier(**params_xgb, 
                                  objective         = 'binary:logistic',                  
                                  predictor         = 'gpu_predictor',
                                  tree_method       = 'gpu_hist',
                                  eval_metric       = 'auc', 
                                  random_state      = 59 
                                 ) 

        model.fit(X_tr, y_tr)

        y_hat_prob = model.predict_proba(X_vl)[:, 1]
        y_hat      = (y_hat_prob >.5).astype(int) 
        
        fpr, tpr, thresholds = metrics.roc_curve(y_vl, y_hat_prob)
                 
        log_loss_     = metrics.log_loss(y_vl, y_hat_prob)                
        f1_score_     = metrics.f1_score(y_vl, y_hat)        
        auc_          = metrics.auc(fpr, tpr)    
                
        msg = '[Fold {}] AUC: {:.5f} - F1: {:.5f} - L. LOSS: {:.5f}'
        print(msg.format(i, auc_, f1_score_, log_loss_))

        df_submission.claim += model.predict_proba(X_ts)[:, 1]/FOLDS
        model_feature_imp   += model.feature_importances_ / FOLDS
        
        X_train_oof_shap[test_idx, :] = np.hstack([X_vl , y_hat.reshape(-1, 1)])
                
        f1.append(f1_score_)
        lloss.append(log_loss_)
        auc.append(auc_)
        
    auc_mean   = np.mean(auc)
    auc_std    = np.std(auc)
    lloss_mean = np.mean(lloss)
    f1_mean    = np.mean(f1)
    
    print('-'*80)
    msg = '[Mean Fold] AUC: {:.5f}(Std:{:.5f}) - F1: {:.5f} - L. LOSS: {:.5f}'
    print(msg.format(auc_mean, auc_std, f1_mean, lloss_mean))
    print('='*80)
    print('')
    
    # Gerar o arquivo de submissão 
    name_file_subm = str(scaler).lower()[:4] + '_' + scale_distribution 
    name_file_subm = 'Data/sumbmission/001_xgb_feature_engineering_' + name_file_subm + '.csv'
    
    df_submission.to_csv(name_file_subm, index=False)
    
    gc.collect()

In [ ]:
# ================================================================================
# Scaler: QuantileTransformer(output_distribution='normal', random_state=0) => NORMAL
# ================================================================================
# [Fold 1] AUC: 0.81068 - F1: 0.79229 - L. LOSS: 0.51150
# [Fold 2] AUC: 0.76741 - F1: 0.73193 - L. LOSS: 0.59884
# [Fold 3] AUC: 0.76947 - F1: 0.74404 - L. LOSS: 0.58815
# [Fold 4] AUC: 0.76998 - F1: 0.73370 - L. LOSS: 0.59284
# [Fold 5] AUC: 0.81071 - F1: 0.79160 - L. LOSS: 0.51071
# --------------------------------------------------------------------------------
# [Mean Fold] AUC: 0.78565(Std:0.02047) - F1: 0.75871 - L. LOSS: 0.56041
# ================================================================================
# 
# CPU times: user 3min 55s, sys: 10.8 s, total: 4min 5s
# Wall time: 3min 59s

<div class="alert alert-info" role="alert">
    
**`NOTA:`**

Como podemos obsevar acima, a média de AUC na validação cruzada foi de 0.78552 e na submissão do kaggle foi uma AUC de 0.80771, sem ajustes nos parametros do XGB.  

## 4.2. Feature importances

Nesta etapa vamos utilizar duas formas de verificar a importância das variáveis para o modelo: 
- As feature importance (feature_importances_) do próprio modelo 
- Shap

### 4.2.1. Feature_importances_ 

In [ ]:
cols_feature    = X.columns.to_list()
cols_feature.append('fe_md_baseline')

df               = pd.DataFrame()
df["Feature"]    = cols_feature
df["Importance"] = model_feature_imp / model_feature_imp.sum()

df.sort_values("Importance", axis=0, ascending=False, inplace=True)

fig, ax = plt.subplots(figsize=(10, 30))
bars = ax.barh(df["Feature"], df["Importance"], 
               height    = 0.8,
               color     = "blue",  # mediumorchid
               edgecolor = "black")

ax.set_title("Feature importances \n", fontsize=30, pad=15)
ax.set_ylabel("Feature name", fontsize=20, labelpad=15)
ax.set_xlabel("Importance", fontsize=15, labelpad=15)
ax.set_yticks(df["Feature"])
ax.set_yticklabels(df["Feature"], fontsize=15)
ax.tick_params(axis="x", labelsize=15)
ax.grid(axis="x")

# Adding labels on top
ax2 = ax.secondary_xaxis('top')
#ax2.set_xlabel("Feature importance", fontsize=20, labelpad=15)
ax2.tick_params(axis="x", labelsize=15)

# Inverting y axis direction so the values are decreasing
plt.gca().invert_yaxis()

Acima podemos observar que algumas das variáveis que criamos estão entre as 25 variáveis mais importantes para o modelo, porém não gosto muito de jeito de analisar a importância das variáveis, vamos utilizar o `Shap` que utilza vários métodos diferentes para encontrar as variáveis importantes. 

## Shap

SHAP (SHapley Additive exPlanations) é uma abordagem teórica de jogos para explicar a saída de qualquer modelo de aprendizado de máquina, ver o  [artigos](https://github.com/slundberg/shap#citations) para detalhes e citações.

In [ ]:
shap.summary_plot(X_train_oof_shap[:,:-1], cols_feature,)

Como podemo observar a ordem das variáveis importantes para o modelo mudarão, agora temos as 4 primeira variáveis que foram criadas como principais, no notebook de feature select vou utilizar o `Shap` para analise da seleção das variáveis. 

Vamos salvar o dataset para o próximo notebook, que será a tunagem de parametros. 

In [ ]:
jb.dump(df2_train,  "Data/pkl/df3_train.pkl.z")
jb.dump(df2_test,  "Data/pkl/df3_test.pkl.z")

gc.collect()

- 